In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
if SETTING["signal_type"] == "raw":

    raise

In [ ]:
feature_x_sample_processed = pd.read_table(
    PATH["feature_x_sample.processed.tsv"], index_col=0
)

In [ ]:
for element_type, df in (
    ("feature", feature_x_sample_processed),
    ("sample", feature_x_sample_processed.T),
):

    element_x_fit_parameter = ccal.fit_skew_t_pdfs(
        df,
        n_job=SETTING["max_n_job"],
        output_file_path=PATH["{}_x_fit_parameter.tsv".format(element_type)],
    )

    element_x_fit_parameter = pd.read_table(
        PATH["{}_x_fit_parameter.tsv".format(element_type)], index_col=0
    )

    for parameter, parameter_values in element_x_fit_parameter.items():

        if parameter_values.unique().size < 2:

            continue

        ccal.plot_distributions(
            (parameter_values,),
            names=(parameter,),
            plot_rug=parameter_values.size < SETTING["plot_rug_max_size"],
            title="{}<br>Distribution of {}".format(
                SETTING["{}_alias".format(element_type)], parameter
            ),
            xaxis_title=parameter,
        )

        parameter_values_sorted = parameter_values.sort_values()

        first_element = parameter_values_sorted.index[0]

        last_element = parameter_values_sorted.index[-1]

        ccal.plot_distributions(
            (df.loc[first_element], df.loc[last_element]),
            names=(first_element, last_element),
            plot_rug=df.shape[1] < SETTING["plot_rug_max_size"],
            title="{}<br>Distributions with Extreme {}".format(
                SETTING["{}_alias".format(element_type)], parameter
            ),
            xaxis_title=SETTING["feature_x_sample_value_name"],
        )

        n_extreme = 3

        for rank in list(range(n_extreme)) + list(range(-n_extreme, 0)):

            ranked_element = parameter_values_sorted.index[rank]

            element_values = df.loc[ranked_element]

            n_data, location, scale, degree_of_freedom, shape = element_x_fit_parameter.loc[
                ranked_element,
                ["N Data", "Location", "Scale", "Degree of Freedom", "Shape"],
            ]

            ccal.plot_context(
                element_values,
                n_data=n_data,
                location=location,
                scale=scale,
                degree_of_freedom=degree_of_freedom,
                shape=shape,
                plot_rug=element_values.dropna().size < SETTING["plot_rug_max_size"],
                title="{} {} Rank {}: {}".format(
                    SETTING["{}_alias".format(element_type)],
                    parameter,
                    rank,
                    ranked_element,
                ),
                xaxis_title=SETTING["feature_x_sample_value_name"],
            )